# Выбор локации для скважины

Нужно решить, где бурить новую скважину.

У нас есть пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

## Оглавление

1. [Изучение общей информации](#step_1)
2. [Предобработка данных](#step_2)
3. [Подготовка к расчёту прибыли](#step_3)
4. [Расчёт прибыли и рисков](#step_4)
5. [Вывод](#step_5)

<a id="step_1"></a>
## 1. Изучение общей информации

Загрузим необходимы библиотеки и подготовим DataFrame.

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
pd.set_option('mode.chained_assignment', None)

In [4]:
df_0.info()
df_0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [5]:
df_1.info()
df_1

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
...,...,...,...,...,...
99995,QywKC,9.535637,-6.878139,1.998296,53.906522
99996,ptvty,-10.160631,-12.558096,5.005581,137.945408
99997,09gWa,-7.378891,-3.084104,4.998651,137.945408
99998,rqwUm,0.665714,-6.152593,1.000146,30.132364


In [6]:
df_2.info()
df_2

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


- id - уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей)

Обработки данных не требуется, переходим к обучению моделей.

<a id="step_2"></a>
## 2. Обучение и проверка модели

Подготовим признаки и целевой признак для обучения модели.

In [7]:
features_0 = df_0.drop(['product', 'id'], axis=1)
target_0 = df_0['product']

features_1 = df_1.drop(['product', 'id'], axis=1)
target_1 = df_1['product']

features_2 = df_2.drop(['product', 'id'], axis=1)
target_2 = df_2['product']

Разделим данные на 2 части (75:25), для обучения и предсказания моделей.

In [8]:
def split(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=12)
    return features_train, features_valid, target_train, target_valid

In [10]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = split(features_0, target_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = split(features_1, target_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = split(features_2, target_2)

Обучим 3 модели и найдем среднее предсказание и значение RMSE.

In [30]:
def rmse_and_mean(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predict = model.predict(features_valid)
    rmse_linreg = mean_squared_error(target_valid, predict)
    print('Среднее линейной регрессии в регионе: {:.3f}'.format(predict.mean()))
    print('RMSE линейной регрессии в регионе: {:.3f}'.format(rmse_linreg**0.5))
    print('')
    return predict

In [32]:
predict_0 = rmse_and_mean(features_train_0, features_valid_0, target_train_0, target_valid_0) # Регион №1

Среднее линейной регрессии в регионе: 92.134
RMSE линейной регрессии в регионе: 37.788



In [33]:
predict_1 = rmse_and_mean(features_train_1, features_valid_1, target_train_1, target_valid_1) # Регион №2

Среднее линейной регрессии в регионе: 69.038
RMSE линейной регрессии в регионе: 0.889



In [34]:
predict_2 = rmse_and_mean(features_train_2, features_valid_2, target_train_2, target_valid_2) # Регион №3

Среднее линейной регрессии в регионе: 95.051
RMSE линейной регрессии в регионе: 39.800



Как видно в регионе №3 самые высокие показатели, дальше проверим подтвердится ли это.

<a id="step_3"></a>
## 3. Подготовка к расчёту прибыли

Подсчетаем основные экономические показатели.

In [27]:
BUDGET = 10000000000
BARELL_1_PRICE = 450000
COUNT_BORELHOLE = 200
BUDGET_FOR_ONE_BOREHOLE = BUDGET / COUNT_BORELHOLE
BREAKEVEN = BUDGET_FOR_ONE_BOREHOLE / BARELL_1_PRICE
print('Точка безубыточности = {:.2f} бареллей или {:.2f} рублей, с 1 вышки.'.format(BREAKEVEN, BUDGET_FOR_ONE_BOREHOLE))

Точка безубыточности = 111.11 бареллей или 50000000.00 рублей, с 1 вышки.


In [28]:
print('Средний объем запасов бареллей в регионе 0 на 1 месторождение = {:.3f}'.format(df_0['product'].mean()))
print('Средний объем запасов бареллей в регионе 1 на 1 месторождение = {:.3f}'.format(df_1['product'].mean()))
print('Средний объем запасов бареллей в регионе 2 на 1 месторождение = {:.3f}'.format(df_2['product'].mean()))

Средний объем запасов бареллей в регионе 0 на 1 месторождение = 92.500
Средний объем запасов бареллей в регионе 1 на 1 месторождение = 68.825
Средний объем запасов бареллей в регионе 2 на 1 месторождение = 95.000


Как видно, средние значения не дотягивают до точки безубыточности, что может показаться, что все регионы убыточны, проверим это далее.

<a id="step_4"></a>
## 4. Расчёт прибыли и рисков 

Применим reset_index к нашим переменным с предсказаниями и правильными ответами, чтобы совпадали индексы.

In [35]:
target_valid_0_pd = pd.Series(target_valid_0).reset_index(drop=True)
predict_0_pd = pd.Series(predict_0).reset_index(drop=True)

target_valid_1_pd = pd.Series(target_valid_1).reset_index(drop=True)
predict_1_pd = pd.Series(predict_1).reset_index(drop=True)

target_valid_2_pd = pd.Series(target_valid_2).reset_index(drop=True)
predict_2_pd = pd.Series(predict_2).reset_index(drop=True)

Теперь перейдем к расчету прибыли и рисков, а также доверительного интервала.

In [48]:
def revenue(target, predict, count):
    predict_sort = predict.sort_values(ascending=False)
    selected = target[predict_sort.index][:count].sum()
    return selected * BARELL_1_PRICE - BUDGET

state = np.random.RandomState(1)

def bootstrap_test(target, predict):
    values = []
    counter = 0
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        predict_subsample = predict[target_subsample.index]
        revenue_result = revenue(target_subsample, predict_subsample, 200)
        values.append(revenue_result)
        if revenue_result <= 0: 
            counter += 1   

    values = pd.Series(values)
    mean = values.mean()
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    return mean, lower, upper, counter/len(values)*100

In [49]:
def bootstrap_results(target, predict, region):
    mean, lower, upper, risk = bootstrap_test(target, predict)
    print('Регион №', region)
    print('Средняя прибыль: {:.2f}'.format(mean))
    print('Доверительный интервал от {:.2f} до {:.2f}'.format(lower, upper))
    print('Риск получения убытка', risk, '%')

In [50]:
bootstrap_results(target_valid_0_pd, predict_0_pd, 1)

Регион № 1
Средняя прибыль: 474839856.18
Доверительный интервал от -93396271.50 до 1001349649.38
Риск получения убытка 4.5 %


In [51]:
bootstrap_results(target_valid_1_pd, predict_1_pd, 2)

Регион № 2
Средняя прибыль: 548945408.18
Доверительный интервал от 147271450.07 до 959382016.59
Риск получения убытка 0.5 %


In [52]:
bootstrap_results(target_valid_2_pd, predict_2_pd, 3)

Регион № 3
Средняя прибыль: 362777386.93
Доверительный интервал от -172542381.64 до 899084351.50
Риск получения убытка 9.6 %


<a id="step_5"></a>
## 5. Вывод

В 1 регионе мы имеем вероятность получить самую большу прибыль, чуть больше 100 млн. руб. из трех регионов, но в то же время имеется риск, в 4.5%, получить и убыток до 93.4 млн. руб.

В 2 регионе самый малый показатель риска, 0.5%, и здесь есть вероятность получить прибыль от 147 до 959 млн. руб. Средний показатель прибыли максимыльный, из всех регионов.

В 3 регионе самые большие риски, 9.6%, и вероятность получить самые большие убытки, в размере 172 млн. руб. Также в этом регионе самый маленький показатель средней прибыли.

Регион 2 является самым привлекательным для освоения. Такой вывод связан с низким риском получить убыток, а также с возможностью получить прибыль от 147 до 960 млн. руб.